In [ ]:
python src/tools/image_slicer.py --image_path 2025_08_05_PhotoRieboR4_g201b201078_f003_259.JPG --overlap_percentage 30 --destination_folder ./slices/ --slice-size 206

In [25]:
slice_path = "slices/2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_004_004_3484_3484.png"

In [ ]:
!nvidia-smi

: 

In [ ]:
import torch
from ultralytics import YOLO

print(f"CUDA доступна: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Название GPU: {torch.cuda.get_device_name(0)}")
    print(f"Количество GPU: {torch.cuda.device_count()}")
    print(f"Текущий GPU device: {torch.cuda.current_device()}")
    device = 'cuda'
else:
    print("CUDA недоступна, используется CPU")
    device = 'cpu'

model_name = "yolo11x"
model = YOLO(f'{model_name}.pt')
model.to(device)
print(f"Модель загружена на: {device}")

: 

In [20]:
# How to specify the threshold for the predictions?
# Передаем device в predict для использования GPU
predictions = model.predict(slice_path, conf=0.1, device=device)
print(f"Количество детекций: {len(predictions[0].boxes.conf)}")
print(f"Используемое устройство: {device}")
print(predictions[0].boxes)


image 1/1 c:\Users\sduai\Desktop\Batyrbayev_Adilet\qCloudy\Projects\PoC\Saiga\slices\2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_004_004_3484_3484.png: 640x640 6 birds, 324.7ms
Speed: 2.6ms preprocess, 324.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
6
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([14., 14., 14., 14., 14., 14.])
conf: tensor([0.4879, 0.2146, 0.1628, 0.1515, 0.1265, 0.1171])
data: tensor([[7.6293e+02, 1.4444e+02, 7.8858e+02, 1.9441e+02, 4.8795e-01, 1.4000e+01],
        [4.5013e+02, 6.8710e+01, 4.6815e+02, 9.9111e+01, 2.1459e-01, 1.4000e+01],
        [4.8737e+02, 2.8446e+02, 5.3031e+02, 3.0364e+02, 1.6284e-01, 1.4000e+01],
        [9.8532e+02, 6.0502e+01, 1.0091e+03, 8.6155e+01, 1.5150e-01, 1.4000e+01],
        [3.3416e+02, 2.5854e+02, 3.5551e+02, 2.8701e+02, 1.2649e-01, 1.4000e+01],
        [7.0392e+02, 2.4229e+02, 7.2850e+02, 2.7271e+02, 1.1715e-01, 1.4000e+01]])
id: None
is_track: False
orig_shape: (1024, 1024)
s

In [17]:
predictions[0].save("predicted_image.png")

'predicted_image.png'

In [8]:
images_folder = "slices/"

In [ ]:
import os
import time

output_folder = f"predicted_images-{model_name}"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

print(f"Обработка изображений на устройстве: {device}")
start_time = time.time()
processed_count = 0
total_detections = 0

# it has to get inside folders in folder slices/ and then predict each image inside it
for folder in os.listdir(images_folder):
    folder_path = f"{images_folder}/{folder}"
    if os.path.isdir(folder_path):
        for image_path in os.listdir(folder_path):
            full_image_path = f"{folder_path}/{image_path}"
            # Используем GPU для предсказаний
            predictions = model.predict(full_image_path, conf=0.1, device=device, verbose=False)
            if len(predictions[0].boxes.conf) > 0:
                predictions[0].save(f"{output_folder}/{image_path}")
                processed_count += 1
                total_detections += len(predictions[0].boxes.conf)
                if processed_count % 10 == 0:  # Показываем прогресс каждые 10 изображений
                    print(f"Обработано: {processed_count} изображений с детекциями")

end_time = time.time()
print(f"\nГотово! Обработано {processed_count} изображений с детекциями")
print(f"Общее количество детекций: {total_detections}")
print(f"Время обработки: {end_time - start_time:.2f} секунд")
print(f"Устройство: {device}")



image 1/1 c:\Users\sduai\Desktop\Batyrbayev_Adilet\qCloudy\Projects\PoC\Saiga\slices\2025_08_05_PhotoRieboR4_g201b201078_f003_259\2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_000_000_0_0.png: 640x640 (no detections), 376.6ms
Speed: 2.6ms preprocess, 376.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\sduai\Desktop\Batyrbayev_Adilet\qCloudy\Projects\PoC\Saiga\slices\2025_08_05_PhotoRieboR4_g201b201078_f003_259\2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_000_001_145_0.png: 640x640 (no detections), 349.7ms
Speed: 2.3ms preprocess, 349.7ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\sduai\Desktop\Batyrbayev_Adilet\qCloudy\Projects\PoC\Saiga\slices\2025_08_05_PhotoRieboR4_g201b201078_f003_259\2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_000_002_290_0.png: 640x640 (no detections), 346.3ms
Speed: 2.2ms preprocess, 346.3ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image

In [ ]:
import multiprocessing as mp
from functools import partial
from tqdm import tqdm
import time
import torch

def process_single_image_gpu(image_path, images_folder, model_path="yolo11x.pt", conf=0.1, device='cuda'):
    """
    Функция для обработки одного изображения на GPU
    Примечание: Для многопроцессорной обработки на GPU рекомендуется использовать batch processing
    """
    try:
        from ultralytics import YOLO
        # Проверяем доступность CUDA в каждом процессе
        if device == 'cuda' and not torch.cuda.is_available():
            device = 'cpu'
            
        model = YOLO(model_path)
        model.to(device)
        
        full_path = f"{images_folder}/{image_path}"
        predictions = model.predict(full_path, conf=conf, device=device, verbose=False)
        
        if len(predictions[0].boxes.conf) > 0:
            output_path = f"predicted_images_gpu/{image_path}"
            predictions[0].save(output_path)
            return image_path, len(predictions[0].boxes.conf), True
        else:
            return image_path, 0, False
    except Exception as e:
        print(f"Ошибка при обработке {image_path}: {e}")
        return image_path, 0, False


In [ ]:
def process_images_gpu_batch(images_folder="slices/", conf=0.1, batch_size=16, model_path="yolo11x.pt"):
    """
    Более эффективная батчевая обработка на GPU
    """
    import torch
    from ultralytics import YOLO
    
    # Проверяем доступность CUDA
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Используется устройство: {device}")
    
    if not os.path.exists("predicted_images_gpu_batch"):
        os.makedirs("predicted_images_gpu_batch")

    # Собираем все пути к изображениям
    image_paths = []
    for folder in os.listdir(images_folder):
        folder_path = f"{images_folder}/{folder}"
        if os.path.isdir(folder_path):
            for image_file in os.listdir(folder_path):
                if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append((f"{folder_path}/{image_file}", image_file))
    
    total_images = len(image_paths)
    print(f"Найдено {total_images} изображений для обработки")
    
    # Загружаем модель один раз
    model = YOLO(model_path)
    model.to(device)
    
    start_time = time.time()
    processed_count = 0
    total_detections = 0
    
    # Обрабатываем изображения батчами
    for i in tqdm(range(0, total_images, batch_size), desc="Обработка батчей"):
        batch_paths = image_paths[i:i + batch_size]
        batch_full_paths = [path for path, _ in batch_paths]
        batch_names = [name for _, name in batch_paths]
        
        try:
            # Предсказание для батча
            results = model.predict(batch_full_paths, conf=conf, device=device, verbose=False)
            
            # Сохраняем результаты
            for result, image_name in zip(results, batch_names):
                if len(result.boxes.conf) > 0:
                    output_path = f"predicted_images_gpu_batch/{image_name}"
                    result.save(output_path)
                    processed_count += 1
                    total_detections += len(result.boxes.conf)
                    
        except Exception as e:
            print(f"Ошибка при обработке батча {i//batch_size + 1}: {e}")
    
    end_time = time.time()
    
    print(f"\nРезультаты батчевой обработки на GPU:")
    print(f"Всего изображений: {total_images}")
    print(f"Изображений с детекциями: {processed_count}")
    print(f"Общее количество детекций: {total_detections}")
    print(f"Время обработки: {end_time - start_time:.2f} секунд")
    print(f"Среднее время на изображение: {(end_time - start_time) / total_images:.3f} секунд")
    print(f"Использованное устройство: {device}")
    
    return processed_count, total_detections

def process_images_multiprocessing_gpu(images_folder="slices/", num_processes=None, conf=0.1):
    """
    Многопроцессорная обработка с поддержкой GPU (для CPU лучше использовать батчевую обработку)
    """
    if not os.path.exists("predicted_images_gpu"):
        os.makedirs("predicted_images_gpu")

    image_files = []
    for folder in os.listdir(images_folder):
        folder_path = f"{images_folder}/{folder}"
        if os.path.isdir(folder_path):
            for f in os.listdir(folder_path):
                if f.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_files.append(f)
    
    total_images = len(image_files)
    print(f"Найдено {total_images} изображений для обработки")
    
    if num_processes is None:
        num_processes = min(mp.cpu_count(), 4)  # Меньше процессов для GPU
    
    print(f"Используется {num_processes} процессов")
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    process_func = partial(process_single_image_gpu, 
                          images_folder=images_folder, 
                          conf=conf,
                          device=device)
    
    start_time = time.time()
    
    # Многопроцессорная обработка
    with mp.Pool(processes=num_processes) as pool:
        results = list(tqdm(pool.imap(process_func, image_files), 
                           total=total_images, 
                           desc="Обработка изображений на GPU"))
    
    end_time = time.time()
    
    # Анализ результатов
    processed_count = sum(1 for _, _, saved in results if saved)
    total_detections = sum(count for _, count, _ in results)
    
    print(f"\nРезультаты многопроцессорной обработки на GPU:")
    print(f"Всего изображений: {total_images}")
    print(f"Изображений с детекциями: {processed_count}")
    print(f"Общее количество детекций: {total_detections}")
    print(f"Время обработки: {end_time - start_time:.2f} секунд")
    print(f"Среднее время на изображение: {(end_time - start_time) / total_images:.2f} секунд")
    
    return results


In [ ]:
# Выбор метода обработки
print("Доступные методы обработки:")
print("1. Батчевая обработка на GPU (рекомендуется для GPU)")
print("2. Многопроцессорная обработка на GPU")
print("3. Обычная последовательная обработка на GPU")

# РЕКОМЕНДУЕМЫЙ способ для GPU - батчевая обработка
print("\n=== БАТЧЕВАЯ ОБРАБОТКА НА GPU (РЕКОМЕНДУЕТСЯ) ===")
if __name__ == "__main__":
    results = process_images_gpu_batch(
        images_folder="slices/",
        conf=0.1,
        batch_size=16,  # Размер батча - настройте под вашу GPU
        model_path=f"{model_name}.pt"
    )

# Альтернативный вариант - многопроцессорная обработка
# (раскомментируйте, если хотите попробовать)
"""
print("\n=== МНОГОПРОЦЕССОРНАЯ ОБРАБОТКА НА GPU ===")
if __name__ == "__main__":
    results = process_images_multiprocessing_gpu(
        images_folder="slices/",
        num_processes=2,  # Меньше процессов для GPU
        conf=0.1
    )
"""
